In [1]:
#sparse optical flow shi-tomashi
import numpy as np
import cv2

cap = cv2.VideoCapture('tennis_clip.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.1,
                       minDistance = 200,
                       blockSize = 25 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 50, 0.003))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
count = 0
while(1):
    count+=1
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    if p1 is None:
        p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        p1, status, error = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

<ipython-input-1-2eb0ec461b4f>:47: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
<ipython-input-1-2eb0ec461b4f>:48: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)


In [2]:
#dense farne back
import numpy as np
import cv2
import numpy as np
import math
cap = cv2.VideoCapture("tennis_clip.mp4")

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
prvs = cv2.bilateralFilter(prvs,25,10,100)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    frame2=np.array(frame2,dtype='float32')
    #frame2 = frame2/2
    #frame2 = frame2*frame2
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, next, cv2.CV_32F, 0.9, 7, 15, 3, 7, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    #rgb = cv2.bilateralFilter(rgb,10,10,100)
    #rgb = rgb*rgb
    gray = cv2.cvtColor(rgb,cv2.COLOR_BGR2GRAY)
    thresh, binary = cv2.threshold(gray,80,255,cv2.THRESH_BINARY)
    cv2.imshow('frame2',rgb)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',rgb)
    prvs = next

cap.release()
cv2.destroyAllWindows()

In [10]:
#Harris corners
import numpy as np
import cv2
import numpy as np
import math
cap = cv2.VideoCapture("tennis_clip.mp4")

ret, frame1 = cap.read()
frame1 = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
prev = frame1
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    frame2=np.array(frame2,dtype='float32')
    kernel = np.ones((5,5), np.uint8)
    gray = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(15,15),0)
    dst = cv2.cornerHarris(gray,9,3,0.08)
    dst = cv2.dilate(dst,kernel,iterations=1)
    flow = cv2.calcOpticalFlowFarneback(prev, dst, cv2.CV_32F, 0.9, 7, 15, 3, 7, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    gray1 = cv2.cvtColor(hsv,cv2.COLOR_HSV2GRAY)
    img = abs(dst-prev)
    prev = dst
    cv2.imshow('dst',gray1)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()

ValueError: could not broadcast input array from shape (496,864) into shape (496)

In [68]:
import numpy as np
import cv2
import numpy as np
import math
cap = cv2.VideoCapture("tennis_clip.mp4")

ret, frame1 = cap.read()
prev = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
while(1):
    ret, frame2 = cap.read()
    frame2=np.array(frame2,dtype='float32')
    frame2 = cv2.GaussianBlur(frame2,(7,7),0)
    #frame2 = cv2.normalize(frame2, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    gray = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    gray = gray[int(gray.shape[0]/5.5):int(gray.shape[0]*4/5), int(gray.shape[1]/5):int(gray.shape[1]*3.5/5)]
    dst = cv2.cornerHarris(gray,9,3,0.04)
    dst = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    sift = cv2.SIFT_create()
    gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    kp1, des1 = sift.detectAndCompute(prev,None)
    kp2, des2 = sift.detectAndCompute(gray,None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.9*n.distance:
            good.append([m])
    # cv.drawMatchesKnn expects list of lists as matches.
    img = cv2.drawMatchesKnn(prev,kp1,gray,kp2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    #img=cv2.drawKeypoints(prev,kp1,prev)
    prev = gray
    cv2.imshow('sift',img)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()

In [11]:
#difference of frames
import numpy as np
import cv2
import numpy as np
import math
cap = cv2.VideoCapture("tennis_clip.mp4")
ret, frame1 = cap.read()
prev = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
prev = cv2.GaussianBlur(prev,(5,5),0)
while(1):
    ret, frame2 = cap.read()
    frame2=cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    frame2 = cv2.GaussianBlur(frame2,(5,5),0)
    diff = abs(frame2-prev)
    diff[diff>100] = 255
    diff[diff<=100] = 0
    diff = cv2.dilate(diff,(5,5))
    prev = frame2
    cv2.imshow('diff',diff)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()